In [1]:
%run transform_utils.ipynb

In [3]:
conn = sqlite3.connect('tubing.db')
cursor = conn.cursor()

In [4]:
# writing the query 
sql_query = """
WITH ProfitChanges AS (
    SELECT
        t.location_id,
        t.date,
        SUM(t.profit) AS daily_sum_profit,
        CASE
            WHEN SUM(t.profit) > 0 THEN 'positive'
            WHEN SUM(t.profit) < 0 THEN 'negative'
            ELSE 'neutral'
        END AS profit_indicator
    FROM
        transactions t
    GROUP BY
        t.location_id, t.date
),
ProfitPercChange AS (
    SELECT
        pc1.location_id,
        pc1.date,
        pc1.daily_sum_profit,
        pc1.profit_indicator,
        ((pc1.daily_sum_profit - IFNULL(pc2.daily_sum_profit, 0)) / IFNULL(pc2.daily_sum_profit, 1)) * 100 AS percent_change_profit
    FROM
        ProfitChanges pc1
    LEFT JOIN
        ProfitChanges pc2 ON pc1.location_id = pc2.location_id AND pc1.date = DATE(pc2.date, '+1 day')
)
SELECT
    w.location_id,
    w.date,
    w.temperature,
    ppc.daily_sum_profit,
    ppc.profit_indicator,
    ppc.percent_change_profit,
    (
        SELECT SUM(daily_sum_profit)
        FROM ProfitChanges
        WHERE location_id = w.location_id AND date BETWEEN DATE(w.date, '-29 days') AND w.date
    ) AS rolling_30_day_sum_profit
FROM
    weather w
LEFT JOIN
    ProfitPercChange ppc ON w.location_id = ppc.location_id AND w.date = ppc.date
ORDER BY
    w.location_id, w.date
LIMIT 50;
"""


In [5]:
cursor.execute(sql_query)

# Fetch the results
results = cursor.fetchall()

# Close the cursor and the connection
cursor.close()
conn.close()

# Print the results
for row in results:
    print(row)

('001', '2019-01-02', 16.88, None, None, None, None)
('001', '2019-01-03', 17.240000000000002, None, None, None, 430.96000000000004)
('001', '2019-01-04', 9.32, None, None, None, 847.86)
('001', '2019-01-05', 20.12, None, None, None, 1128.02)
('001', '2019-01-06', 17.240000000000002, None, None, None, 1544.0)
('001', '2019-01-07', 14.9, None, None, None, 1956.2)
('001', '2019-01-09', 15.8, None, None, None, 2282.73)
('001', '2019-01-10', 20.12, None, None, None, 2663.2400000000002)
('001', '2019-01-11', 19.58, None, None, None, 3061.75)
('001', '2019-01-12', 15.44, None, None, None, 3459.12)
('001', '2019-01-13', 16.52, None, None, None, 3895.41)
('001', '2019-01-14', 6.800000000000001, None, None, None, 4385.72)
('001', '2019-01-15', 15.98, None, None, None, 4520.650000000001)
('001', '2019-01-16', 8.779999999999998, None, None, None, 4825.380000000001)
('001', '2019-01-17', 12.38, None, None, None, 5003.100000000001)
('001', '2019-01-18', 8.240000000000002, None, None, None, 5166.210